Import das bibliotecas

In [1]:
from retinaface import RetinaFace
import pandas as pd     

Lendo o arquivo do conjunto de dados FairFace

In [28]:
df=pd.read_csv("../data/raw/fairface_label_train.csv")

Separando cada etnia para um dataframe diferente

In [3]:
df_east_asian=df[df['race']=='East Asian']
df_indian=df[df['race']=='Indian']
df_black=df[df['race']=='Black']
df_white=df[df['race']=='White']
df_middle_eastern=df[df['race']=='Middle Eastern']
df_latino_hispanic=df[df['race']=='Latino_Hispanic']
df_southeast_asian=df[df['race']=='Southeast Asian']

Foi avaliado cada etnia separadamente

In [4]:
df_race=df_southeast_asian

Foi criada uma nova coluna no dataframe

In [5]:
df_race['faces_retina']=pd.NA

Foi aplicado o modelo Retina Face e apresesentado a quantidade de faces que foi possível detectar

In [ ]:
i=0
for index, row in df_race.iterrows():  
    image_path_short = row['file']
    image_path_long = '../data/raw/' + image_path_short
    try:
        resp = RetinaFace.detect_faces(image_path_long)
        df_race.loc[index, 'faces_retina'] = len(resp)
    except:
        df_race.loc[index, 'faces_retina'] = 'não foi detectado nenhum rosto'
    if i>2000:
        break
    i+=1

O resultado foi salvo na pasta data/interim

In [6]:
df_race.to_csv('../data/interim/retina_face_southeast_asian.csv')

Para avaliação, foram lidos todos os conjuntos

In [29]:
df_black=pd.read_csv("../data/interim/retina_face_black.csv")
df_east_asian=pd.read_csv("../data/interim/retina_face_east_asian.csv")
df_indian=pd.read_csv("../data/interim/retina_face_indian.csv")
df_latino_hispanic=pd.read_csv("../data/interim/retina_face_latino_hispanic.csv")
df_middle_eastern=pd.read_csv("../data/interim/retina_face_middle_eastern.csv")
df_southeast_asian=pd.read_csv("../data/interim/retina_face_southeast_asian.csv")
df_white=pd.read_csv("../data/interim/retina_face_white.csv")
list_dfs=[df_black, df_east_asian, df_indian, df_latino_hispanic, df_middle_eastern, df_southeast_asian, df_white]

Foi avaliado a quantidade de imagens que o modelo foi aplicado

In [31]:
for df in list_dfs:
    print('Quantidade de valores não vazios no dataframe da raça', df['race'][0],':', df['faces_retina'].count())

Quantidade de valores não vazios no dataframe da raça Black : 2002
Quantidade de valores não vazios no dataframe da raça East Asian : 2002
Quantidade de valores não vazios no dataframe da raça Indian : 2002
Quantidade de valores não vazios no dataframe da raça Latino_Hispanic : 2002
Quantidade de valores não vazios no dataframe da raça Middle Eastern : 2002
Quantidade de valores não vazios no dataframe da raça Southeast Asian : 2002
Quantidade de valores não vazios no dataframe da raça White : 2002


Foi avaliado a quantidade de imagens que não foi possível detectar o rosto

In [27]:
for df in list_dfs:
    print('Dataframe de Erros na Raça ', df['race'][0], ':')
    print(df[df['faces_retina']==0])
    print("\n")

Dataframe de Erros na Raça  Black :
Empty DataFrame
Columns: [Unnamed: 0, file, age, gender, race, service_test, faces_retina]
Index: []


Dataframe de Erros na Raça  East Asian :
Empty DataFrame
Columns: [Unnamed: 0, file, age, gender, race, service_test, faces_retina]
Index: []


Dataframe de Erros na Raça  Indian :
Empty DataFrame
Columns: [Unnamed: 0, file, age, gender, race, service_test, faces_retina]
Index: []


Dataframe de Erros na Raça  Latino_Hispanic :
Empty DataFrame
Columns: [Unnamed: 0, file, age, gender, race, service_test, faces_retina]
Index: []


Dataframe de Erros na Raça  Middle Eastern :
Empty DataFrame
Columns: [Unnamed: 0, file, age, gender, race, service_test, faces_retina]
Index: []


Dataframe de Erros na Raça  Southeast Asian :
Empty DataFrame
Columns: [Unnamed: 0, file, age, gender, race, service_test, faces_retina]
Index: []


Dataframe de Erros na Raça  White :
Empty DataFrame
Columns: [Unnamed: 0, file, age, gender, race, service_test, faces_retina]
Inde

Foi instalado a biblioteca fairlearn para avaliar o viés no modelo

In [26]:
!pip install fairlearn

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     -------------------------------------- 60.8/60.8 kB 814.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/234.1 kB ? eta -:--:--
   ---------------------------------------- 234.1/234.1 kB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.0 MB 30.4 MB/s eta 0:00:01
   ------- -------------------------------- 2.2/11.0 MB 27.7 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/11.0 MB 25.1 MB/s eta 0:00:01
   ------------- -------------------------- 3.7/11.0 MB 26.3 MB/s eta 0:00:01
   ----------------- ---------------------- 4.7/11.0 MB 21.6 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.0 MB 22.6 MB/s eta 0:00:01
   ------------------------- -------------- 7.1/11.0 MB 23.9 MB/s eta 0:00:01
   -----


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Todas os diferentes dataframes foram unidos novamente em um só

In [32]:
# Concatenate all DataFrames
combined_df = pd.concat(list_dfs)

# Sort the combined DataFrame by the index column
df_retina = combined_df.sort_values(by='Unnamed: 0')

# Reset the index if you want a clean integer index
df_retina.reset_index(drop=True, inplace=True)

df_retina.head()

df_retina['face_detected'] = df_retina['faces_retina'] >= 1

O seguinte código calcula as métricas para avaliação de viés

In [47]:
from fairlearn.metrics import MetricFrame, true_positive_rate, true_negative_rate, false_positive_rate, false_negative_rate, selection_rate

df=df_retina

# Verificar se a coluna 'service_test' representa a verdade sobre a presença de um rosto
df['face_present'] = df['service_test']  # Supondo que 'service_test' é True se um rosto está presente
# Definir as colunas para avaliação
y_true = df['face_present']
y_pred = df['face_detected']
sensitive_features = df[['gender', 'race']]


# Calcular métricas usando Fairlearn
metrics = {
    'true_positive_rate': true_positive_rate,
    'false_positive_rate': false_positive_rate,
    'true_negative_rate': true_negative_rate,
    'false_negative_rate': false_negative_rate,
    'selection_rate': selection_rate
}

metric_frame = MetricFrame(metrics=metrics, y_true=y_true, y_pred=y_pred, sensitive_features=sensitive_features)

# Paridade Demográfica: selection_rate
demographic_parity = metric_frame.by_group['selection_rate']
print("\nParidade Demográfica (Taxa de Seleção por Raça):")
print(demographic_parity)

# Igualdade de Oportunidade: true_positive_rate
equalized_opportunity = metric_frame.by_group['true_positive_rate']
print("\nIgualdade de Oportunidade (Taxa de Verdadeiros Positivos por Raça):")
print(equalized_opportunity)

# Igualdade de Chance: true_positive_rate e false_negative_rate
equalized_chance = metric_frame.by_group[['true_positive_rate', 'false_positive_rate']]
print("\nIgualdade de Chance (Taxas de Verdadeiros Positivos e Falsos Negativos por Raça):")
print(equalized_chance)

# Compute true positive rate and false positive rate by group
true_positive_rates = metric_frame.by_group['true_positive_rate']
false_positive_rates = metric_frame.by_group['false_positive_rate']

# Compare true positive rate and false positive rate for each group
predictive_parity = (true_positive_rates - false_positive_rates).abs()

print("Predictive Parity (Absolute Difference between True Positive Rate and False Positive Rate by Race):")
print(predictive_parity)

# Compute false positive rate and false negative rate by group
false_positive_rates = metric_frame.by_group['false_positive_rate']
false_negative_rates = metric_frame.by_group['false_negative_rate']

# Compare false positive rate and false negative rate for each group
treatment_equality = (false_positive_rates - false_negative_rates).abs()

print("Equality of Treatment (Absolute Difference between False Positive Rate and False Negative Rate by Race):")
print(treatment_equality)

C:\Users\DELL\AppData\Local\Temp\ipykernel_15704\3884262700.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['face_present'] = df['service_test']  # Supondo que 'service_test' é True se um rosto está presente
